In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://10.80.102.113:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1543246556252)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7cabb261


In [2]:
spark.version

res1: String = 2.3.1


# Kaggle Dataset Load

In [3]:
val input = sc.textFile("Scorecard.csv").map(line =>line.split(","))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at <console>:25


### Create a header index of the dataset

In [4]:
val header = sc.parallelize(input.take(1))

header: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[3] at parallelize at <console>:27


In [5]:
val headerIndex = header.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

headerIndex: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[6] at zipWithIndex at <console>:29


In [6]:
//headerIndex.coalesce(1).saveAsTextFile("index")

Originally, I exported and imported the data. Not sure why.

In [7]:
// header.map(_.mkString(",")).coalesce(1).saveAsTextFile("header")

In [8]:
// val headerLoad = sc.textFile("header.csv")

In [9]:
// val headerList= headerLoad.flatMap(line => line.split(","))

In [10]:
// val headerIndex = headerList.zipWithIndex

In [11]:
val idxSat = headerIndex.lookup("SAT_AVG")(0).toInt

idxSat: Int = 60


# DoE Dataset Load

In [12]:
val in2016 = sc.textFile("MERGED2016_17_PP.csv").map(line =>line.split(","))

val marked2016 = in2016.map(_.mkString(",")).map(x => x + ",2016")

val in2015 = sc.textFile("MERGED2015_16_PP.csv").map(line =>line.split(","))

val marked2015 = in2015.map(_.mkString(",")).map(x => x + ",2015")

val in2014 = sc.textFile("MERGED2014_15_PP.csv").map(line =>line.split(","))

val marked2014 = in2014.map(_.mkString(",")).map(x => x + ",2014")

val in2013 = sc.textFile("MERGED2013_14_PP.csv").map(line =>line.split(","))

val marked2013 = in2016.map(_.mkString(",")).map(x => x + ",2013")

val in2012 = sc.textFile("MERGED2012_13_PP.csv").map(line =>line.split(","))

val marked2012 = in2012.map(_.mkString(",")).map(x => x + ",2012")

val in2011 = sc.textFile("MERGED2011_12_PP.csv").map(line =>line.split(","))

val marked2011 = in2011.map(_.mkString(",")).map(x => x + ",2011")

val in2010 = sc.textFile("MERGED2010_11_PP.csv").map(line =>line.split(","))

val marked2010 = in2010.map(_.mkString(",")).map(x => x + ",2010")

val in2009 = sc.textFile("MERGED2009_10_PP.csv").map(line =>line.split(","))

val marked2009 = in2009.map(_.mkString(",")).map(x => x + ",2009")

val in2008 = sc.textFile("MERGED2008_09_PP.csv").map(line =>line.split(","))

val marked2008 = in2008.map(_.mkString(",")).map(x => x + ",2008")

val in2007 = sc.textFile("MERGED2007_08_PP.csv").map(line =>line.split(","))

val marked2007 = in2007.map(_.mkString(",")).map(x => x + ",2007")

in2016: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[11] at map at <console>:25
marked2016: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[13] at map at <console>:27
in2015: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[16] at map at <console>:29
marked2015: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[18] at map at <console>:31
in2014: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[21] at map at <console>:33
marked2014: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[23] at map at <console>:35
in2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[26] at map at <console>:37
marked2013: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[28] at map at <console>:39
in2012: org.apache.spark.rdd.RDD[Array[String]] = MapPart...

In [ ]:
// Test the year concat function. 
// marked2016.coalesce(1).saveAsTextFile("2016test")


## Compile the 10 year RDD.

In [36]:
val scoreCard10Year = sc.union(marked2016, marked2015, marked2014, marked2013, marked2012, marked2011, marked2010, marked2009, marked2008, marked2007).map(line => line.split(","))

scoreCard10Year: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[72] at map at <console>:45


### Create a header index of the dataset

In [46]:
val header2016 = sc.parallelize(in2016.take(1))

val header2016Index = header2016.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2016Index.coalesce(1).saveAsTextFile("index2016") //Textfiles only made once

val header2007 = sc.parallelize(in2007.take(1))

val header2007Index = header2007.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2007Index.coalesce(1).saveAsTextFile("index2007") // Textfiles only made once

header2016: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[79] at parallelize at <console>:30
header2016Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[82] at zipWithIndex at <console>:32
header2007: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[83] at parallelize at <console>:36
header2007Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[86] at zipWithIndex at <console>:38


## Count the rows of the kaggle compiled dataset

In [44]:

/*
input.count()

input.map( x => x(1)).distinct().count()

val year = input.map( x => x(1730))

year.take(1100).foreach(println)

It looks like there are a lot of blank lines in there early on in the dataset. 

val notBlankYear = input.filter(x => x(1730) != "")

notBlankYear.count()

notBlankYear.map(x => x(1730)).take(20)

notBlankYear.filter(x => x(1730) contains "2013").count()
*/


## Count of rows for the compiled DoE Scorecard

In [15]:
scoreCard10Year.count()

res7: Long = 73552


# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years

In [67]:
val ust = scoreCard10Year.filter(x => x(1) contains "174914")

ust: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[101] at filter at <console>:47


In [68]:
ust.take(2)

res30: Array[Array[String]] = Array()


In [61]:
val ust2013 = input.filter( x => x(4) contains "University of St Thomas").filter(x => x(5) contains "Saint Paul").filter( x => x(1730) contains "2013")

ust2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[96] at filter at <console>:27


In [62]:
val ustHeader = header.union(ust)

ustHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[97] at union at <console>:31


# Create an index of the header 

Really had a tough time figuring out how to get the header loaded as just a string. Loading it from the original source results in type RDD[Array[String]], so I ended up using a CSV file that I outputed from the original input. Ended up just needing a **flatMap** to keep it in the correct format. 

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [12]:
val mnColleges = input.filter( x => x(5) contains "MN").filter( x => x(1730) contains "2013")

mnColleges: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[17] at filter at <console>:27


# Regional Schools  
This RDD contains schools from the upper-midwest (Minnesota, Wisconsin, North Dakota, South Dakota, Iowa, Nebraska). This RDD will be used for regional comparisons. 

In [43]:
val upMidWestColleges = scoreCard10Year.filter( x => x(19) contains "Plains")

upMidWestColleges: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[78] at filter at <console>:47


# TextFile Saves

In [ ]:
headerIndex.foreach {
    case(field, ind) => println(s"$field is $ind")
}

In [ ]:
ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

In [ ]:
headerIndex.map(_.mkString(",")).coalesce(1).saveAsTextFile("headerIndex")

In [ ]:
mnColleges.map(_.mkString(",")).coalesce(1).saveAsTextFile("Minnesota")

In [14]:
val regionalHeader = header.union(upMidWestColleges)

regionalHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[19] at union at <console>:31


In [58]:
regionalHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("regionalHeader")